In [123]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

In [124]:
daal_df = pd.read_csv('../output/cleaned_daal2.csv')
daal_df.head(25)

,CDB ID,OSC Account Name,Country Name,FY24 Account Segmentation,DAAL NB
0,7425027,1976,PE,Longtail,1976
1,523314,37135,DO,Longtail,37135
2,1080194,44988,AR,Longtail,44258
3,7561885,50624412411,CR,Longtail,50624412411
4,7434745,ALLIANZ TECHNOLOGY OF AMERICA,US,Segment 2,ALLIANZ
5,6863506,HARRIS COUNTY,US,Segment 1,COUNTY OF HARRIS TX
6,7239941,ISSAM IMPORTACAO E EXPORTACAO EIRELI,BR,Longtail,ISSAM IMPORTACAO E EXPORTACAO EIRELI
7,7234731,RICHLAND COUNTY EMERGENCY SERVICES,US,Longtail,RICHLAND COUNTY EMERGENCY SERVICES
8,11670,SUPERIOR COURT OF CALIFORNIAVENTURA COUNTY,US,Longtail,COUNTY OF VENTURA CA
9,1508091,TERADATA,US,Segment 1,TERADATA


In [125]:
input_df = pd.read_csv('../output/cleaned_input.csv')
input_df

,CDB_ID,Party_NumberĿ,Account(Customer_Name),Country_Code_Only
0,1,230401,FLEXERA SOFTWARE,US
1,2,230402,TIANJIN SWCX TECHNOLOGY,CN
2,3,230403,SUNGARD AVAILABILITY SERVICES,IN
3,4,230404,CLINSE LABS,IN
4,5,230405,MYLAN,US
...,...,...,...,...
5437,37164,235838,JHU PRESS,US
5438,37191,235839,LG CHEM KOREA,KR
5439,37218,235840,FPT TELECOM INTERNATIONAL,VN
5440,37245,235841,VIRGIN MANAGEMENT,GB


## Steps to process the uploaded input file and daal file
    1. firstly read input file and daal file, store them in 2 different dataframes.
    2. Iterate on 'input_file_df' and for each record from input file, find top 3 matching records from "daal_file_df" using fuzzyMatchPercentage. (threshold 80% above)
    3. While finding top 3 matching records :
        3.1. filter entire 'daal_file_df' dataframe accoring to common country names.
        3.2. if out of 3 top matching records, if at any point found 100% fuzzMatched , skip processing of remaining daal file (time will be reduced)
        3.3. 
    4. create 'new_df' with 2 columns as 'company name' (from input file) and 'similar company name' (from daal file), after done with step 2 and 3.
    5. now calculate other 7 attributes i.e. values of simple ratio, partial ratio, token set ratio, ngram, charact_matching_percentage etc. and add these columns to 'new_df'
    6. pass this 'new_df' to ML model for prediction of 'output' column as 0/1. And store only records with output = 1 (same) into final_df passed to flask application.

In [126]:
from fuzzywuzzy import process, fuzz

# Function to filter daal_df based on common country code
def filter_daal_df(country_code, reference_df):
    filtered_df = reference_df[reference_df['Country Name'] == country_code]
    # print(filtered_df)
    return filtered_df

# fuzzy ratio function
def calculate_fuzzy_ratio(str1, str2):
    return fuzz.ratio(str1, str2)

# Function to find top 3 matching records
def find_top_matches(row, reference_df, threshold=75):
    # Convert NaN values to empty strings in both columns
    query_str = str(row['Account(Customer_Name)']) if not pd.isna(row['Account(Customer_Name)']) else ''
    
    # Calculate fuzzy ratio for each record in reference_df and filter based on threshold
    reference_df['Fuzz Ratio'] = reference_df['OSC Account Name'].apply(lambda x: calculate_fuzzy_ratio(query_str, str(x)))
    
    # Check if a perfect match (fuzzy ratio of 100) is found **** (so we can skip remaining daal file search)
    perfect_match = reference_df[reference_df['Fuzz Ratio'] == 100].head(1)
    
    if not perfect_match.empty:
        # If a perfect match is found, skip further processing and return the result
        return pd.DataFrame({
            'Account(Customer_Name)': [query_str],
            'OSC Account Name': perfect_match['OSC Account Name'].tolist(),
            'Score': [100]
        })
    
    # Filter based on threshold
    filtered_matches = reference_df[reference_df['Fuzz Ratio'] > threshold]
    
    # Extract top 3 matches
    top_matches = filtered_matches.sort_values(by='Fuzz Ratio', ascending=False).head(3)
    
    # Return the result as a DataFrame
    result_df = pd.DataFrame({
        'Account(Customer_Name)': [query_str] * len(top_matches),
        'OSC Account Name': top_matches['OSC Account Name'].tolist(),
        'Score': top_matches['Fuzz Ratio'].tolist()
    })
    
    return result_df


# Function row-wise on the first 30 records of input_df
matched_df_list = []
for index, row in input_df.head(50).iterrows():
    # Filter daal_df based on the common country code
    country_code = row['Country_Code_Only']
    filtered_daal_df = filter_daal_df(country_code, daal_df)
    # print(filtered_daal_df)

    # Find top matches for the current row
    matches_df = find_top_matches(row, filtered_daal_df)
    matched_df_list.append(matches_df)

# Concatenate the list of DataFrames into a single DataFrame
matched_result_df = pd.concat(matched_df_list, ignore_index=True)

# Display the new DataFrame
print("Matched Result DataFrame (First 30 records from input_df):")
matched_result_df



Matched Result DataFrame (First 30 records from input_df):


,Account(Customer_Name),OSC Account Name,Score
0,FLEXERA SOFTWARE,FLEXERA SOFTWARE,100.0
1,TIANJIN SWCX TECHNOLOGY,TIANJIN ENEN TECHNOLOGY,83.0
2,TIANJIN SWCX TECHNOLOGY,TIANJIN AIMA TECHNOLOGY,83.0
3,TIANJIN SWCX TECHNOLOGY,TIANJIN SWCX ELECTRONIC TECHNOLOGY,81.0
4,CLINSE LABS,PINE LABS,80.0
5,MYLAN,MYLAN,100.0
6,COCHIN SHIPYARD,COCHIN SHIPYARD,100.0
7,KOTAK MAHINDRA GENERAL INSURANCE,KOTAK MAHINDRA GENERAL INSURANCE,100.0
8,GUJARAT GAS,GUJARAT GAS,100.0
9,RAJCOMP INFORMATION SERVICES,RAJCOMP INFORMATION SERVICE,98.0


In [127]:
matched_result_df = matched_result_df.drop("Score", axis=1)
matched_result_df

,Account(Customer_Name),OSC Account Name
0,FLEXERA SOFTWARE,FLEXERA SOFTWARE
1,TIANJIN SWCX TECHNOLOGY,TIANJIN ENEN TECHNOLOGY
2,TIANJIN SWCX TECHNOLOGY,TIANJIN AIMA TECHNOLOGY
3,TIANJIN SWCX TECHNOLOGY,TIANJIN SWCX ELECTRONIC TECHNOLOGY
4,CLINSE LABS,PINE LABS
5,MYLAN,MYLAN
6,COCHIN SHIPYARD,COCHIN SHIPYARD
7,KOTAK MAHINDRA GENERAL INSURANCE,KOTAK MAHINDRA GENERAL INSURANCE
8,GUJARAT GAS,GUJARAT GAS
9,RAJCOMP INFORMATION SERVICES,RAJCOMP INFORMATION SERVICE


In [128]:
# calculating the ratio values as attributes
# 'Simple Ratio', 'partial Ratio', 'Token Set Ratio', 'word match percentage', 'last word match', 'character_matching_percentage', 'ngrams'
from feature_extraction import calculate_simple_ratio, calculate_partial_ratio, calculate_token_set_ratio, calculate_word_matching_percentage, last_word_match, character_matching_percentage, calculate_ngrams_similarity

# Extracting features (ratios) from the fuzzy matching scores
matched_result_df['Simple Ratio'] = matched_result_df.apply(lambda row: calculate_simple_ratio(row['Account(Customer_Name)'], row['OSC Account Name']), axis=1) 
matched_result_df['partial Ratio'] = matched_result_df.apply(lambda row: calculate_partial_ratio(row['Account(Customer_Name)'], row['OSC Account Name']), axis=1) 
matched_result_df['Token Set Ratio'] = matched_result_df.apply(lambda row: calculate_token_set_ratio(row['Account(Customer_Name)'], row['OSC Account Name']), axis=1) 
matched_result_df['word match percentage'] = matched_result_df.apply(lambda row: calculate_word_matching_percentage(row['Account(Customer_Name)'], row['OSC Account Name']), axis=1) 
matched_result_df['last word match'] = matched_result_df.apply(lambda row: last_word_match(row['Account(Customer_Name)'], row['OSC Account Name']), axis=1) 
matched_result_df['last word match'] = matched_result_df['last word match'].astype(int)
matched_result_df['character_matching_percentage'] = matched_result_df.apply(lambda row: character_matching_percentage(row['Account(Customer_Name)'], row['OSC Account Name']), axis=1) 
matched_result_df['ngrams'] = matched_result_df.apply(lambda row: calculate_ngrams_similarity(row['Account(Customer_Name)'], row['OSC Account Name']), axis=1) 

# Display the DataFrame with predictions
print("DataFrame with Predictions:")
matched_result_df.head(10)


DataFrame with Predictions:


,Account(Customer_Name),OSC Account Name,Simple Ratio,partial Ratio,Token Set Ratio,word match percentage,last word match,character_matching_percentage,ngrams
0,FLEXERA SOFTWARE,FLEXERA SOFTWARE,100,100,100,100.000000,1,100.000000,1.000000
1,TIANJIN SWCX TECHNOLOGY,TIANJIN ENEN TECHNOLOGY,83,83,88,50.000000,1,82.608696,0.680000
2,TIANJIN SWCX TECHNOLOGY,TIANJIN AIMA TECHNOLOGY,83,83,88,50.000000,1,82.608696,0.629630
3,TIANJIN SWCX TECHNOLOGY,TIANJIN SWCX ELECTRONIC TECHNOLOGY,81,70,100,75.000000,1,38.235294,0.687500
4,CLINSE LABS,PINE LABS,80,89,80,33.333333,1,0.000000,0.500000
5,MYLAN,MYLAN,100,100,100,100.000000,1,100.000000,1.000000
6,COCHIN SHIPYARD,COCHIN SHIPYARD,100,100,100,100.000000,1,100.000000,1.000000
7,KOTAK MAHINDRA GENERAL INSURANCE,KOTAK MAHINDRA GENERAL INSURANCE,100,100,100,100.000000,1,100.000000,1.000000
8,GUJARAT GAS,GUJARAT GAS,100,100,100,100.000000,1,100.000000,1.000000
9,RAJCOMP INFORMATION SERVICES,RAJCOMP INFORMATION SERVICE,98,100,98,50.000000,0,96.428571,0.962963


In [129]:
# training svm model on 'cleaned_training_dataset' csv file having newly added more extra features all 16 
df = pd.read_csv('../output/cleaned_training_dataset.csv')
# df

In [130]:
# for training model, we need to split the dataframe and also its features
from sklearn.model_selection import train_test_split
# Separate features and target variable
features = [ 'Simple Ratio', 'partial Ratio', 'Token Set Ratio', 'word match percentage', 'last word match', 'character_matching_percentage', 'ngrams']
xn = df[features] #independent features (input columns)
yn = df['Target'] #dependent feature (output column)
Xn_train, Xn_test, yn_train, yn_test = train_test_split(xn, yn, test_size=0.3, random_state=42)

In [131]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Customizing Kernel Parameters: For both RBF and polynomial kernels, there are additional parameters you can tune, such as gamma for RBF
# Example with RBF kernel and customized gamma value = 0.001
svm_model_rbf_custom = SVC(kernel='poly', gamma=0.001)  # Try different gamma values, we got 0.001 as best gamma value
svm_model_rbf_custom.fit(Xn_train, yn_train)

yn_pred_rbf_custom = svm_model_rbf_custom.predict(Xn_test)

accuracy_rbf_custom = accuracy_score(yn_test, yn_pred_rbf_custom)
report_rbf_custom = classification_report(yn_test, yn_pred_rbf_custom)
print(f"Accuracy (RBF Kernel - Customized Gamma): {accuracy_rbf_custom}")
# print("Classification Report (RBF Kernel - Customized Gamma):\n", report_rbf_custom)

Accuracy (RBF Kernel - Customized Gamma): 0.7593582887700535


In [132]:
# for ML model prediction we have to pass only 7columns attribute values of ratios
# skipping first 2 columns here:
temp_result_df = matched_result_df[[ 'Simple Ratio', 'partial Ratio', 'Token Set Ratio', 'word match percentage', 'last word match', 'character_matching_percentage', 'ngrams']]

# Predict using the ML model
temp_result_df['output'] = svm_model_rbf_custom.predict(temp_result_df)
matched_result_df['output'] = temp_result_df['output']
temp_result_df.head(10)

,Simple Ratio,partial Ratio,Token Set Ratio,word match percentage,last word match,character_matching_percentage,ngrams,output
0,100,100,100,100.000000,1,100.000000,1.000000,0
1,83,83,88,50.000000,1,82.608696,0.680000,0
2,83,83,88,50.000000,1,82.608696,0.629630,0
3,81,70,100,75.000000,1,38.235294,0.687500,1
4,80,89,80,33.333333,1,0.000000,0.500000,1
5,100,100,100,100.000000,1,100.000000,1.000000,0
6,100,100,100,100.000000,1,100.000000,1.000000,0
7,100,100,100,100.000000,1,100.000000,1.000000,0
8,100,100,100,100.000000,1,100.000000,1.000000,0
9,98,100,98,50.000000,0,96.428571,0.962963,1


In [133]:
matched_result_df

,Account(Customer_Name),OSC Account Name,Simple Ratio,partial Ratio,Token Set Ratio,word match percentage,last word match,character_matching_percentage,ngrams,output
0,FLEXERA SOFTWARE,FLEXERA SOFTWARE,100,100,100,100.000000,1,100.000000,1.000000,0
1,TIANJIN SWCX TECHNOLOGY,TIANJIN ENEN TECHNOLOGY,83,83,88,50.000000,1,82.608696,0.680000,0
2,TIANJIN SWCX TECHNOLOGY,TIANJIN AIMA TECHNOLOGY,83,83,88,50.000000,1,82.608696,0.629630,0
3,TIANJIN SWCX TECHNOLOGY,TIANJIN SWCX ELECTRONIC TECHNOLOGY,81,70,100,75.000000,1,38.235294,0.687500,1
4,CLINSE LABS,PINE LABS,80,89,80,33.333333,1,0.000000,0.500000,1
5,MYLAN,MYLAN,100,100,100,100.000000,1,100.000000,1.000000,0
6,COCHIN SHIPYARD,COCHIN SHIPYARD,100,100,100,100.000000,1,100.000000,1.000000,0
7,KOTAK MAHINDRA GENERAL INSURANCE,KOTAK MAHINDRA GENERAL INSURANCE,100,100,100,100.000000,1,100.000000,1.000000,0
8,GUJARAT GAS,GUJARAT GAS,100,100,100,100.000000,1,100.000000,1.000000,0
9,RAJCOMP INFORMATION SERVICES,RAJCOMP INFORMATION SERVICE,98,100,98,50.000000,0,96.428571,0.962963,1


In [135]:
# # Taking only the records with 'output' = 1 (i.e. same company name) into resultant_df with only 3 feilds
# filtered_df = matched_result_df[matched_result_df['output'] == 1]

# # Display the final DataFrame
# print("Filtered DataFrame:")
# filtered_df[['Account(Customer_Name)', 'OSC Account Name', 'output']]